In [4]:
#%matplotlib inline
import os, sys
#sys.path.append('/Users/hhsieh/anaconda3/envs/astroconda/lib/python3.6/site-packages')
#sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData, ImageFileCollection
import numpy as np
import math, glob, os, bz2, subprocess, sys
import cosmics_py3
#import sqlite3
#from sqlite3 import Error
import scipy.signal as signal
import scipy.ndimage as ndimage
import datetime


In [5]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('Directory {:s} successfully created.'.format(path))
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def decompress_file_bzip2(filename):
    cmd = ['bzip2','-d',filename]
    process = subprocess.call(cmd)
    return None

def compress_file_fpack(filename):
    fpack_command  = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    delete_command = '/bin/rm'
    fpack_cmd = [fpack_command,filename]
    process = subprocess.call(fpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('fpack command failed.')
    return None

def decompress_file_fpack(filename):
    funpack_command = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    delete_command  = '/bin/rm'
    funpack_cmd = [funpack_command,filename]
    process = subprocess.call(funpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('funpack command failed.')
    return None

def decompress_directory_bzip2(file_path):
    print('>>> Starting decompression of bz2 files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for bz2_file in sorted(glob.glob('*.bz2')):
        decompress_file_bzip2(bz2_file)
    print('>>> Decompression of bz2 files complete.')
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fits_file in sorted(glob.glob('*.fits')):
        compress_file_fpack(fits_file)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fz_file in sorted(glob.glob('*.fz')):
        decompress_file_fpack(fz_file)
    print('>>> Decompression of fz files complete.')
    return None


#def get_date_id():
#    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
#    # return: YYYYMMDD date id
#    for raw_mef_file in sorted(glob.glob('*.fits')):
#        fits_date = raw_mef_file[1:9]
#    return fits_date

def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed (regardless of current compression state)
    # return: YYYYMMDD date id
    fits_date = ''
    if len(glob.glob('*.fits')) > 0:
        for raw_mef_file in sorted(glob.glob('*.fits')):
            fits_date = raw_mef_file[1:9]
    elif len(glob.glob('*.fits.fz')) > 0:
        for raw_mef_file in sorted(glob.glob('*.fits.fz')):
            fits_date = raw_mef_file[1:9]
    elif len(glob.glob('*.fits.bz2')) > 0:
        for raw_mef_file in sorted(glob.glob('*.fits.bz2')):
            fits_date = raw_mef_file[1:9]
    return fits_date

def create_stats_files(fits_date,imgtype):
    for extid in range(2,10):
        ext_stats = 'n{:s}.{:s}.{:02d}.stats'.format(fits_date,imgtype,extid+1)
        outputfile = open(ext_stats,'w')
        outputfile.write('# Extension {:02d}              NPIX        MEAN     STDDEV         MIN         MAX\n'.format(extid+1))
        outputfile.close()
    return None
        
def compile_stats_files(fits_date,imgtype):
    print('Starting image statistics output...')
    output_stats_filename = 'n{:s}.{:s}.stats'.format(fits_date,imgtype)
    output_stats_file = open(output_stats_filename,'w')
    cmd1 = '/bin/rm'
    for stats_filename in glob.glob('*.??.stats'):
        stats_file = open(stats_filename,'r')
        for line in stats_file:
            output_stats_file.write(line)
        stats_file.close()
        cmd = [cmd1,stats_filename]
        process = subprocess.call(cmd)
    output_stats_file.close()
    print('>>> Image statistics output complete.')
    return None

def split_extensions(imgtype):
    # split 12-element GMOS multi-extension FITS files into individual elements
    # return: dimensions of extensions; also writes individual extension files to working directory
    for raw_mef_file in glob.glob('*.fits'):
        hdulist = fits.open(raw_mef_file)
        print('Splitting {:s}...'.format(raw_mef_file))
        fits_date = raw_mef_file[1:9]
        fits_id   = raw_mef_file[11:14]
        hdr1 = getheader(raw_mef_file,0)
        for extid in range(2,10):
            extension = hdulist[extid+1].data
            hdr2 = hdulist[extid+1].header
            radecsys = hdr2['RADECSYS']
            hdr2['RADESYSA'] = radecsys
            del hdr2['RADECSYS']
            imstat_npix = extension.size
            imstat_min = np.min(extension)
            imstat_max = np.max(extension)
            imstat_mean = np.mean(extension)
            imstat_stdev = np.std(extension)
            ext_filename = 'n' + fits_date + '.' + fits_id + '.{:02d}.fits'.format(extid+1)
            outputfilename = 'n{:s}.{:s}.{:02d}.stats'.format(fits_date,imgtype,extid+1)
            outputfile = open(outputfilename,'a')
            outputfile.write('{:s}   {:>8d}    {:>8.2f}    {:>7.2f}    {:>8.2f}    {:>8.2f}\n'.format(ext_filename,imstat_npix,imstat_mean,imstat_stdev,imstat_min,imstat_max))
            outputfile.close()
            dimensions = extension.shape
            dimension1 = dimensions[0]
            dimension2 = dimensions[1]
            hdr = hdr1 + hdr2
            fits.writeto(ext_filename,extension,hdr)
        hdulist.close()
    return dimension1, dimension2
    print('>>> Multi-extension fits file splitting complete.')
    return None
    
def remove_wrong_dimensions(dimension1,dimension2):
    # identify bias files with correct binning and remove others
    # return: none; retains bias files with correct binning and removes others
    for raw_mef_file in glob.glob('*.fits'):
        hdulist = fits.open(raw_mef_file)
        extension01 = hdulist[1].data
        bias_dimensions = extension01.shape
        bias_dimension1 = bias_dimensions[0]
        bias_dimension2 = bias_dimensions[1]
        hdulist.close()
        delete_file = '/bin/rm'
        if (bias_dimension1 != dimension1) or (bias_dimension2 != dimension2):
            delete_cmd = [delete_file,raw_mef_file]
            process = subprocess.call(delete_cmd)
    print('>>> Bias frame selection complete.')
    return None


def add_fits_header_line(fits_file,param_name,param_value,param_comment):
    # Add line to FITS header
    hdulist = fits.open(fits_file)
    data = hdulist[0].data
    hdr = hdulist[0].header
    hdr[param_name] = (param_value,param_comment)
    fits.writeto(fits_file,data,hdr,overwrite=True)
    return None

def overscan_and_trim():
    # Do overscan correction and trim images in current directory
    print('Starting overscan correction and trimming...')
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        ot_fits_file = fits_file[0:16] + '.ot.fits'
        fits_data = CCDData.read(fits_file,unit=u.adu)
        file_ext = fits_file[14:16]
        # Overscan correction
        if ((file_ext == '03') or (file_ext=='05') or (file_ext=='07') or (file_ext=='09')):
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[257:288,1:2112]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        else:
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[1:32,1:2112]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        # Image trimming
        if ((file_ext == '03') or (file_ext=='07')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[1:256,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        elif ((file_ext == '04') or (file_ext == '08')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:282,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        elif ((file_ext == '05') or (file_ext =='09')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[7:256,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        else: # file_ext == '02' or file_ext == '10'
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:288,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        ot_fits_data.write(ot_fits_file)
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)
    print('>>> Overscan correction and trimming complete.')
    return None

def bias_median_combine(dateid):
    print('Starting median combination of bias frames...')
    for extid in range(2,10):
        bias_list = []
        ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
        output_filename = 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
        for fits_file in sorted(glob.glob(ext_file_format)):
            fits_data = CCDData.read(fits_file)
            bias_list.append(fits_data)
            delete_file = '/bin/rm'
            delete_cmd = [delete_file,fits_file]
            process = subprocess.call(delete_cmd)
        master_bias = cp.combine(bias_list,method='median')
        master_bias.write(output_filename)
    print('>>> Median combination of bias frames complete.')
    return None

def bias_correct(dateid,cwd_raw_bias):
    print('Starting bias subtraction...')
    delete_cmd = '/bin/rm'
    for extid in range(2,10):
        bias_filename = cwd_raw_bias + 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
        ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
        for fits_file in sorted(glob.glob(ext_file_format)):
            fits_data = CCDData.read(fits_file)
            bias_data = CCDData.read(bias_filename)
            fits_date_imageid = fits_file[0:13]
            output_filename = fits_date_imageid + '.{:02d}.otz.fits'.format(extid+1)
            bias_corrected_data = cp.subtract_bias(fits_data,bias_data,add_keyword={'zerocorr':True,'calstat':'OTZ'})
            bias_corrected_data.write(output_filename)
            delete_file = [delete_cmd,fits_file]
            process = subprocess.call(delete_file)
    print('>>> Bias subtraction complete.')
    return None

def concatenate_gmos_amps():
    print('>>> Starting adjacent amp area concatenation...')
    delete_cmd = '/bin/rm'
    for fits_file in sorted(glob.glob('*.03.otz.fits')):
        file_prefix = fits_file[0:13]
        ext1_filename = file_prefix + '.03.otz.fits'
        ext2_filename = file_prefix + '.04.otz.fits'
        hdr = getheader(ext1_filename,0)
        hdulist1 = fits.open(ext1_filename)
        ext1_data = hdulist1[0].data
        hdulist2 = fits.open(ext2_filename)
        ext2_data = hdulist2[0].data
        hdulist1.close()
        hdulist2.close()
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip1.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        delete_file = [delete_cmd,ext1_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext2_filename]
        process = subprocess.call(delete_file)

        ext1_filename = file_prefix + '.05.otz.fits'
        ext2_filename = file_prefix + '.06.otz.fits'
        ext3_filename = file_prefix + '.07.otz.fits'
        ext4_filename = file_prefix + '.08.otz.fits'
        hdr = getheader(ext1_filename,0)
        hdulist1 = fits.open(ext1_filename)
        ext1_data = hdulist1[0].data
        hdulist2 = fits.open(ext2_filename)
        ext2_data = hdulist2[0].data
        hdulist3 = fits.open(ext3_filename)
        ext3_data = hdulist3[0].data
        hdulist4 = fits.open(ext4_filename)
        ext4_data = hdulist4[0].data
        hdulist1.close()
        hdulist2.close()
        hdulist3.close()
        hdulist4.close()
        chip1_data = np.hstack((ext1_data,ext2_data,ext3_data,ext4_data))
        output_filename = file_prefix + '.chip2.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        delete_file = [delete_cmd,ext1_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext2_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext3_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext4_filename]
        process = subprocess.call(delete_file)

        ext1_filename = file_prefix + '.09.otz.fits'
        ext2_filename = file_prefix + '.10.otz.fits'
        hdr = getheader(ext1_filename,0)
        hdulist1 = fits.open(ext1_filename)
        ext1_data = hdulist1[0].data
        hdulist2 = fits.open(ext2_filename)
        ext2_data = hdulist2[0].data
        hdulist1.close()
        hdulist2.close()
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip3.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        delete_file = [delete_cmd,ext1_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext2_filename]
        process = subprocess.call(delete_file)
    print('>>> Adjacent amp area concatenation complete.')
    return None

def flatfield_correction(base_dir,data_dir,flatfield_chip1_file,flatfield_chip2_file,flatfield_chip3_file):
    print('Starting flatfield correction...')
    for fits_file in sorted(glob.glob('*.chip1.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(base_dir+data_dir+'/'+flatfield_chip1_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip1.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.03.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.04.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','n/a','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','n/a','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip1_file,'Flatfield frame used')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip2.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(base_dir+data_dir+'/'+flatfield_chip2_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip2.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.05.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.06.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3',fits_date_imageid+'.bias.07.fits','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4',fits_date_imageid+'.bias.08.fits','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip2_file,'Flatfield frame used')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip3.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(base_dir+data_dir+'/'+flatfield_chip3_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip3.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.09.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.10.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','n/a','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','n/a','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip3_file,'Flatfield frame used')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    print('>>> Flatfield correction complete.')
    return None

def cosmicray_cleaning():
    print('Starting cosmic ray correction...')
    for fits_file in sorted(glob.glob('*.chip1.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip2.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip3.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)
        
    print('>>> Cosmic ray correction complete.')
    return None

def move_reduced_data(rawfitsdir,rawbiasdir,flatdir,path_stats):
    move_file = 'mv'
    os.chdir(rawfitsdir)
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        move_cmd = [move_file,fits_file,flatdir]
        process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        move_cmd = [move_file,stats_file,path_stats]
        process = subprocess.call(move_cmd)
    os.chdir(rawbiasdir)
    for fits_file in sorted(glob.glob('*.bias.??.fits')):
        move_cmd = [move_file,fits_file,flatdir]
        process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        move_cmd = [move_file,stats_file,path_stats]
        process = subprocess.call(move_cmd)
    print('>>> Processed data moved to output directory.')
    return None

def move_reduced_science_data(rawfitsdir,flatdir,path_stats):
    move_file = 'mv'
    os.chdir(rawfitsdir)
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        move_cmd = [move_file,fits_file,flatdir]
        process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        move_cmd = [move_file,stats_file,path_stats]
        process = subprocess.call(move_cmd)
    #os.chdir(rawbiasdir)
    #for fits_file in sorted(glob.glob('*.bias.??.fits')):
    #    move_cmd = [move_file,fits_file,flatdir]
    #    process = subprocess.call(move_cmd)
    #for stats_file in sorted(glob.glob('*.stats')):
    #    move_cmd = [move_file,stats_file,path_stats]
    #    process = subprocess.call(move_cmd)
    print('>>> Processed data moved to output directory.')
    return None

def move_reduced_bias_data(rawbiasdir,flatdir,path_stats):
    move_file = 'mv'
    #os.chdir(rawfitsdir)
    #for fits_file in sorted(glob.glob('*.otzfc.fits')):
    #    move_cmd = [move_file,fits_file,flatdir]
    #    process = subprocess.call(move_cmd)
    #for stats_file in sorted(glob.glob('*.stats')):
    #    move_cmd = [move_file,stats_file,path_stats]
    #    process = subprocess.call(move_cmd)
    os.chdir(rawbiasdir)
    if os.path.exists(flatdir):
        for fits_file in sorted(glob.glob('*.bias.??.fits')):
            move_cmd = [move_file,fits_file,flatdir]
            process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        move_cmd = [move_file,stats_file,path_stats]
        process = subprocess.call(move_cmd)
    print('>>> Processed bias data moved to output directory.')
    return None

def move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1,flatfield_chip2,flatfield_chip3):
    move_file = 'mv'
    os.chdir(base_dir + data_dir)
    move_cmd = [move_file,flatfield_chip1,path_procfits]
    process = subprocess.call(move_cmd)
    move_cmd = [move_file,flatfield_chip2,path_procfits]
    process = subprocess.call(move_cmd)
    move_cmd = [move_file,flatfield_chip3,path_procfits]
    process = subprocess.call(move_cmd)
    return None

    
print('Done.')


Done.


In [6]:
def fix_gain_rdnoise_gmosn_hamamatsu():
    # Fix gain and read noise values in FITS headers
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        with fits.open(fits_file) as hdulist:
            data = hdulist[0].data
            hdr = hdulist[0].header
        hdr['GAIN'] = 1.63
        hdr['RDNOISE'] = 4.14
        fits.writeto(fits_file,data,hdr,overwrite=True)
    print('>>> Gain and read noise updated in image headers.')
    return None

def fix_gain_rdnoise_gmoss_hamamatsu():
    # Fix gain and read noise values in FITS headers
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        with fits.open(fits_file) as hdulist:
            data = hdulist[0].data
            hdr = hdulist[0].header
        hdr['GAIN'] = 1.83
        hdr['RDNOISE'] = 3.98
        fits.writeto(fits_file,data,hdr,overwrite=True)
    print('>>> Gain and read noise updated in image headers.')
    return None

def convert_to_long_target_desig(short_desig):
    long_desig = short_desig
    if short_desig == '133P': long_desig = '133P/1996 N2 (Elst-Pizarro 1)'
    if short_desig == '176P': long_desig = '176P/1999 RE70 (LINEAR 52)'
    if short_desig == '238P': long_desig = '238P/2005 U1 (Read 3)'
    if short_desig == '259P': long_desig = '259P/2008 R1 (Garradd 4)'
    if short_desig == '288P': long_desig = '288P/2006 VW139 (300163)'
    if short_desig == '2006 VW139': long_desig = '288P/2006 VW139 (300163)'
    if short_desig == '313P': long_desig = '313P/2014 S4 (Gibbs 16)'
    if short_desig == '324P': long_desig = '324P/2010 R2 (La Sagra 4)'
    if short_desig == '358P': long_desig = '358P/2012 T1 (PanSTARRS 11)'
    return long_desig

def fix_gmos_header_info(instr_id,filter_id):
    # Fix header information for PDS submission
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        with fits.open(fits_file) as hdulist:
            #hdulist = fits.open(fits_file)
            data = hdulist[0].data
            hdr = hdulist[0].header

        # Change keywords for CDi_j parameters (conflict with other WCS keywords)
        cd1_1,cd1_1_comment = hdr['CD1_1'],hdr.comments['CD1_1']
        cd1_2,cd1_2_comment = hdr['CD1_2'],hdr.comments['CD1_2']
        cd2_1,cd2_1_comment = hdr['CD2_1'],hdr.comments['CD2_1']
        cd2_2,cd2_2_comment = hdr['CD2_2'],hdr.comments['CD2_2']
        hdr['x_CD1_1'] = (cd1_1,cd1_1_comment)
        hdr['x_CD1_2'] = (cd1_2,cd1_2_comment)
        hdr['x_CD2_1'] = (cd2_1,cd2_1_comment)
        hdr['x_CD2_2'] = (cd2_2,cd2_2_comment)
        del hdr['CD1_1']
        del hdr['CD1_2']
        del hdr['CD2_1']
        del hdr['CD2_2']
        
        # Change keyword for EPOCH parameter
        target_epoch,target_epoch_comment = hdr['EPOCH'],hdr.comments['EPOCH']
        hdr['TRGEPOCH'] = (target_epoch,target_epoch_comment)
        del hdr['EPOCH']
        
        # Remove RADVEL parameter (not correctly populated)
        del hdr['RADVEL']
        
        # Change EXPTIME value to integer format
        exptime = int(round(hdr['EXPTIME']))
        hdr['EXPTIME'] = exptime
        
        # Add keyword for full object designation
        object_name = hdr['OBJECT']
        object_desig = convert_to_long_target_desig(object_name)
        hdr['OBJDESIG'] = object_desig

        # Add central filter wavelength keyword
        if instr_id == 'gmosn':
            if filter_id == 'g':   hdr['CENTWAVE'] = 475
            elif filter_id == 'r': hdr['CENTWAVE'] = 630
            elif filter_id == 'i': hdr['CENTWAVE'] = 780
        elif instr_id == 'gmoss':
            if filter_id == 'g':   hdr['CENTWAVE'] = 475
            elif filter_id == 'r': hdr['CENTWAVE'] = 630
            elif filter_id == 'i': hdr['CENTWAVE'] = 780
        
        # Remove checksums
        if 'CHECKSUM' in hdr: del hdr['CHECKSUM']
        if 'DATASUM'  in hdr: del hdr['DATASUM']
        
        fits.writeto(fits_file,data,hdr,overwrite=True,checksum=True)
        
    print('>>> Header information fixed for PDS submission.')

    return None

def find_flatfield_files(base_dir,data_dir,instr_id,filter_id):
    ff_chip1_filename,ff_chip2_filename,ff_chip3_filename = 'none','none','none'
    os.chdir(base_dir + data_dir)
    decompress_directory_fpack(base_dir + data_dir)
    flatfield_filename_pattern = 'n*.' + instr_id + '.twiskyflat.' + filter_id + '*.fits'
    if len(glob.glob(flatfield_filename_pattern)) != 3:
        print('Incorrect number of flatfield files found.')
    else:
        for flatfield_file in sorted(glob.glob(flatfield_filename_pattern)):
            if flatfield_file[-10:-5] == 'chip1': ff_chip1_filename = flatfield_file
            if flatfield_file[-10:-5] == 'chip2': ff_chip2_filename = flatfield_file
            if flatfield_file[-10:-5] == 'chip3': ff_chip3_filename = flatfield_file
    return ff_chip1_filename,ff_chip2_filename,ff_chip3_filename
                
def trim_gmoss_data_with_bad_amp(data_path):
    # Trim GMOS-S data with bad amp 5 in current directory
    print('Trimming GMOS-S data with bad amp #5...')
    os.chdir(data_path)
    for fits_file_fz in sorted(glob.glob('n*.???.chip2.otzfc.fits.fz')):
        decompress_file_fpack(fits_file_fz)
        fits_file = fits_file_fz[:-3]
        new_fits_file = fits_file[0:13] + '.chip2a.otzfc.fits'        
        fits_data = CCDData.read(fits_file,unit=u.adu)
        trimmed_fits_data = cp.trim_image(fits_data,fits_section='[251:1012,1:2046]')        
        trimmed_fits_data.write(new_fits_file)
        compress_file_fpack(fits_file)
        compress_file_fpack(new_fits_file)
    print('>>> Trimming GMOS-S data with bad amp #5 complete.')
    return None

print('Done.')


Done.


In [4]:
def process_bias_data(path_rawbias,instr_id,date_id,reduction_log):
    print('\n{:s} - Processing bias-frame data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    reduction_log.write('\n{:s} - Processing bias-frame science data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    
    # decompress raw bias fits files
    print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Decompressing, filtering, and splitting bias files...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    os.chdir(path_rawbias)
    decompress_directory_bzip2(path_rawbias)       # decompress downloaded GMOS files
    decompress_directory_fpack(path_rawbias)       # decompress previously fpacked GMOS files
    #remove_wrong_dimensions(dim1,dim2)       # find and retain bias frames matching data file binning
    #date_id = get_date_id()
    create_stats_files(date_id,'bias')       # create files to record image statistics
    split_extensions('bias')                 # split bias MEFs into individual FITS files
    if instr_id == 'gmosn':
        fix_gain_rdnoise_gmosn_hamamatsu()
    elif instr_id == 'gmoss':
        fix_gain_rdnoise_gmoss_hamamatsu()
    compile_stats_files(date_id,'bias')      # collect image stats of bias files together and delete individual files
    
    # perform overscan corrections and trim data for bias frames
    os.chdir(path_rawbias)
    overscan_and_trim()
    bias_median_combine(date_id)
    
    print('\n{:s} - Processing bias-frame data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    reduction_log.write('\n{:s} - Processing bias-frame science data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawbias))
    return reduction_log


def process_science_data(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log):
    print('\n{:s} - Processing {:s}-band science data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))
    reduction_log.write('\n{:s} - Processing {:s}-band science data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))
    
    flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename = find_flatfield_files(base_dir,data_dir,instr_id,filter_id)
    
    if flatfield_chip1_filename != 'none' and flatfield_chip2_filename != 'none' and flatfield_chip3_filename != 'none':
        if not os.path.isdir(path_procfits): os.mkdir(path_procfits)
        if not os.path.isdir(path_stats):    os.mkdir(path_stats)
        # Process raw science fits files
        print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        reduction_log.write('{:s} - Decompressing and splitting data files...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        os.chdir(path_rawfits)
        decompress_directory_bzip2(path_rawfits)       # decompress downloaded GMOS files
        decompress_directory_fpack(path_rawfits)       # decompress previously fpacked GMOS files
        date_id = get_date_id()
        create_stats_files(date_id,'scidata.{:s}'.format(filter_id))    # create files to record image statistics
        dim1, dim2 = split_extensions('scidata') # split science MEFs into individual FITS files
        if instr_id == 'gmosn':
            fix_gain_rdnoise_gmosn_hamamatsu()
        elif instr_id == 'gmoss':
            fix_gain_rdnoise_gmoss_hamamatsu()
        compile_stats_files(date_id,'scidata.{:s}'.format(filter_id))   # collect image stats of science files together and delete individual files
            
        # Process science frames
        os.chdir(path_rawfits)
        overscan_and_trim()                                                    # Perform overscan correction and trim data
        bias_correct(date_id,path_rawbias)                                      # Perform bias correction
        concatenate_gmos_amps()                                                # Join science frames
        flatfield_correction(base_dir,data_dir,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)  # Do flatfield corrections
        cosmicray_cleaning()                                                   # Do cosmic ray cleaning
        #fix_gmos_header_info(instr_id,filter_id)
    
        ## Clean up data
        #move_reduced_data(path_rawfits,path_rawbias,path_procfits,path_stats)
        ##move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)
        #compress_directory_fpack(base_dir+data_dir)
        #compress_directory_fpack(path_rawfits)
        #compress_directory_fpack(path_rawbias)
        #compress_directory_fpack(path_procfits)
        print('{:s} - Processing science data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        reduction_log.write('{:s} - Processing science data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
            
    else:
        print('{:s} - Appropriate flatfield images for {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
        reduction_log.write('{:s} - Appropriate flatfield images for {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
            
    return reduction_log


def clean_up_science_data(path_rawfits,path_procfits,path_stats):
    # Clean up data
    move_reduced_science_data(path_rawfits,path_procfits,path_stats)
    compress_directory_fpack(path_rawfits)
    return None

def clean_up_common_data(base_dir,data_dir,path_rawbias,path_procfits,path_stats):
    # Clean up data
    move_reduced_bias_data(path_rawbias,path_procfits,path_stats)
    compress_directory_fpack(base_dir+data_dir)
    compress_directory_fpack(path_rawbias)
    if os.path.exists(path_procfits): compress_directory_fpack(path_procfits)
    return None


def process_images(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log):
    print('\n{:s} - Processing {:s}-band science data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))
    reduction_log.write('\n{:s} - Processing {:s}-band science data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))

    flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename = find_flatfield_files(base_dir,data_dir,instr_id,filter_id)

    if flatfield_chip1_filename != 'none' and flatfield_chip2_filename != 'none' and flatfield_chip3_filename != 'none':
        if not os.path.isdir(path_procfits): os.mkdir(path_procfits)
        if not os.path.isdir(path_stats):    os.mkdir(path_stats)
        # Process raw science fits files
        print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        reduction_log.write('{:s} - Decompressing and splitting data files...\n')
        os.chdir(path_rawfits)
        decompress_directory_bzip2(path_rawfits)       # decompress downloaded GMOS files
        decompress_directory_fpack(path_rawfits)       # decompress previously fpacked GMOS files
        date_id = get_date_id()
        create_stats_files(date_id,'scidata')    # create files to record image statistics
        dim1, dim2 = split_extensions('scidata') # split science MEFs into individual FITS files
        if instr_id == 'gmosn':
            fix_gain_rdnoise_gmosn_hamamatsu()
        elif instr_id == 'gmoss':
            fix_gain_rdnoise_gmoss_hamamatsu()
        compile_stats_files(date_id,'scidata')   # collect image stats of science files together and delete individual files

        # decompress raw bias fits files
        print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        reduction_log.write('{:s} - Decompressing, filtering, and splitting bias files...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        os.chdir(path_rawbias)
        decompress_directory_bzip2(path_rawbias)       # decompress downloaded GMOS files
        decompress_directory_fpack(path_rawbias)       # decompress previously fpacked GMOS files
        remove_wrong_dimensions(dim1,dim2)       # find and retain bias frames matching data file binning
        create_stats_files(date_id,'bias')       # create files to record image statistics
        split_extensions('bias')                 # split bias MEFs into individual FITS files
        if instr_id == 'gmosn':
            fix_gain_rdnoise_gmosn_hamamatsu()
        elif instr_id == 'gmoss':
            fix_gain_rdnoise_gmoss_hamamatsu()
        compile_stats_files(date_id,'bias')      # collect image stats of bias files together and delete individual files
    
        # perform overscan corrections and trim data for bias frames
        os.chdir(path_rawbias)
        overscan_and_trim()
        bias_median_combine(date_id)

        # Process science frames
        os.chdir(path_rawfits)
        overscan_and_trim()                                                    # Perform overscan correction and trim data
        bias_correct(date_id,path_rawbias)                                      # Perform bias correction
        concatenate_gmos_amps()                                                # Join science frames
        flatfield_correction(base_dir,data_dir,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)  # Do flatfield corrections
        cosmicray_cleaning()                                                   # Do cosmic ray cleaning
        #fix_gmos_header_info(instr_id,filter_id)
    
        # Clean up data
        move_reduced_data(path_rawfits,path_rawbias,path_procfits,path_stats)
        #move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)
        compress_directory_fpack(base_dir+data_dir)
        compress_directory_fpack(path_rawfits)
        compress_directory_fpack(path_rawbias)
        compress_directory_fpack(path_procfits)
        if instr_id == 'gmoss':
            trim_gmoss_data_with_bad_amp(path_procfits)
        print('{:s} - Processing science data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        reduction_log.write('{:s} - Processing science data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        
    else:
        print('{:s} - Appropriate flatfield images for {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        
        reduction_log.write('{:s} - Appropriate flatfield images for {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),path_rawfits))        

    return None


print('Done.')


Done.


In [5]:
def process_gemini_science_data(base_dir):
    os.chdir(base_dir)
    for data_dir in sorted(glob.glob('ut*_gemini?/')):
    
        if data_dir[-2:-1] == 'N':   instr_id = 'gmosn'
        elif data_dir[-2:-1] == 'S': instr_id = 'gmoss'
        else: instr_id = 'not recognized'

        if instr_id == 'gmosn' or instr_id == 'gmoss':
            os.chdir(base_dir + data_dir)
            reduction_logfile = base_dir + data_dir + 'log_reduction_{:s}_{:s}_{:s}.txt'.format(data_dir[2:10],instr_id,datetime.datetime.today().strftime('%Y%m%d_%H%M%S'))
            path_rawbias  = base_dir + data_dir + 'rawfits_bias/'
            path_procfits = base_dir + data_dir + 'flatfits_pyt/'
            path_stats    = base_dir + data_dir + 'stats/'
            if not os.path.exists(path_procfits):
                with open(reduction_logfile,'w') as reduction_log:
                    if os.path.exists(path_rawbias):
                        if len(os.listdir(path_rawbias)) > 0:
                            if len(glob.glob('rawfits_?/')) > 0:
                                print('\n{:s} - Starting processing of data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                                reduction_log.write('\n{:s} - Starting processing of data in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                                os.chdir(base_dir + data_dir)
                                for dir_rawfits in sorted(glob.glob('rawfits_?/')):
                                    os.chdir(base_dir + data_dir + dir_rawfits)
                                    date_id = get_date_id()
                                reduction_log = process_bias_data(path_rawbias,instr_id,date_id,reduction_log)
                                os.chdir(base_dir + data_dir)
                                for dir_rawfits in sorted(glob.glob('rawfits_?/')):
                                    filter_id = dir_rawfits[-2:-1]
                                    path_rawfits  = base_dir + data_dir + dir_rawfits
                                    reduction_log = process_science_data(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id,reduction_log)
                                    clean_up_science_data(path_rawfits,path_procfits,path_stats)
                                clean_up_common_data(base_dir,data_dir,path_rawbias,path_procfits,path_stats)
                                print('\n{:s} - Processing of data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                                reduction_log.write('{:s} - Processing of data in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                            else:
                                print('{:s} - Raw science data in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                                reduction_log.write('{:s} - Raw science data in {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                        else:
                            print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                            reduction_log.write('{:s} - Bias directory in {:s} is empty.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                    else:
                        print('{:s} - Bias directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                        reduction_log.write('{:s} - Bias directory in {:s} not found.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
            else:
                print('{:s} - Science data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        else:
            print('{:s} - Directory name format not recognized for {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
    return None
    

In [10]:
process_gemini_science_data('/volumes/Fantom12a/BackupData/TargetedObjects/MBCs/P2020O1/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/TargetedObjects/MBCs/P2019A4/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2019B-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2020A-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2020A-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2020B-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2020B-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2021A-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2021A-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2021B-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2021B-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2022A-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022A-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2022B-Q-307/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-LP-104/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-Q-111/')
process_gemini_science_data('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/')
#process_gemini_science_data('/users/hhsieh/data/')

print('Done.')


2023-02-26 08:53:37 - Science data in ut20200813_geminiN/ already processed.
2023-02-26 08:53:37 - Science data in ut20200814_geminiN/ already processed.
2023-02-26 08:53:37 - Science data in ut20200822_geminiN/ already processed.
2023-02-26 08:53:37 - Science data in ut20200922_geminiN/ already processed.
2023-02-26 08:53:37 - Science data in ut20190404_geminiN/ already processed.
2023-02-26 08:53:37 - Science data in ut20190424_geminiN/ already processed.
2023-02-26 08:53:37 - Science data in ut20190803_geminiS/ already processed.
2023-02-26 08:53:37 - Science data in ut20190804_geminiS/ already processed.
2023-02-26 08:53:37 - Science data in ut20190805_geminiS/ already processed.
2023-02-26 08:53:37 - Science data in ut20190806_geminiS/ already processed.
2023-02-26 08:53:37 - Science data in ut20190831_geminiS/ already processed.
2023-02-26 08:53:37 - Science data in ut20190920_geminiS/ already processed.
2023-02-26 08:53:37 - Science data in ut20190929_geminiS/ already processed.

>>> Decompression of bz2 files complete.
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230213_geminiS/rawfits_bias/...
>>> Decompression of fz files complete.
Splitting S20230113S0261.fits...
Splitting S20230113S0262.fits...
Splitting S20230113S0263.fits...
Splitting S20230113S0264.fits...
Splitting S20230113S0265.fits...
>>> Gain and read noise updated in image headers.
Starting image statistics output...
>>> Image statistics output complete.
Starting overscan correction and trimming...


>>> Overscan correction and trimming complete.
Starting median combination of bias frames...
>>> Median combination of bias frames complete.

2023-02-26 08:54:08 - Processing bias-frame data in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230213_geminiS/rawfits_bias/ complete.

2023-02-26 08:54:08 - Processing r-band science data in ut20230213_geminiS/...
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230213_geminiS/...
>>> Decompression of fz files complete.

2023-02-26 08:54:11 - Decompressing and splitting data files...
>>> Starting decompression of bz2 files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230213_geminiS/rawfits_r/...
>>> Decompression of bz2 files complete.
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230213_geminiS/rawfits_r/...
>>> Decompression of fz f

>>> Overscan correction and trimming complete.
Starting bias subtraction...
>>> Bias subtraction complete.
>>> Starting adjacent amp area concatenation...
>>> Adjacent amp area concatenation complete.
Starting flatfield correction...
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead

21715 cosmic pixels (21715 new)
Iteration 2
4291 cosmic pixels (1836 new)
Iteration 3
2807 cosmic pixels (401 new)
Iteration 4
2589 cosmic pixels (137 new)
Cosmic ray cleaning for n20230213.220.chip2.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 15 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
21205 cosmic pixels (21205 new)
Iteration 2
4188 cosmic pixels (1735 new)
Iteration 3
2742 cosmic pixels (314 new)
Iteration 4
2578 cosmic pixels (124 new)
Cosmic ray cleaning for n20230213.221.chip2.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 18 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
20639 cosmic pixels (20639 new)
Iteration 2
4068 cosmic pixels (1611 new)
Iteration 3
2791 cosmic pixels (373 new)
Iteration 4
2620 cosmic pixels (148 new)
Cosmic ray cleaning for n20230213.222.chip2.otzf.fits
Detecting saturated star

...

>>> Overscan correction and trimming complete.
Starting median combination of bias frames...
>>> Median combination of bias frames complete.

2023-02-26 09:29:17 - Processing bias-frame data in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230219_geminiS/rawfits_bias/ complete.

2023-02-26 09:29:17 - Processing r-band science data in ut20230219_geminiS/...
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230219_geminiS/...
>>> Decompression of fz files complete.

2023-02-26 09:29:20 - Decompressing and splitting data files...
>>> Starting decompression of bz2 files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230219_geminiS/rawfits_r/...
>>> Decompression of bz2 files complete.
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230219_geminiS/rawfits_r/...
>>> Decompression of fz f

>>> Overscan correction and trimming complete.
Starting bias subtraction...
>>> Bias subtraction complete.
>>> Starting adjacent amp area concatenation...
>>> Adjacent amp area concatenation complete.
Starting flatfield correction...
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead

Building mask of saturated stars ...
We have 3 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
4504 cosmic pixels (4504 new)
Iteration 2
720 cosmic pixels (649 new)
Iteration 3
160 cosmic pixels (113 new)
Iteration 4
52 cosmic pixels (14 new)
Cosmic ray cleaning for n20230219.019.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 2 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
4590 cosmic pixels (4590 new)
Iteration 2
737 cosmic pixels (672 new)
Iteration 3
131 cosmic pixels (109 new)
Iteration 4
44 cosmic pixels (32 new)
Cosmic ray cleaning for n20230219.020.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 3 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
4378 cosmic pixels (4378 new)
Iteration 2
763 cosmic pixels (693 new)
Iteration 3
140 cosmic pixels (107 new)


>>> Overscan correction and trimming complete.
Starting median combination of bias frames...
>>> Median combination of bias frames complete.

2023-02-26 09:36:43 - Processing bias-frame data in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230226_geminiS/rawfits_bias/ complete.

2023-02-26 09:36:43 - Processing r-band science data in ut20230226_geminiS/...
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230226_geminiS/...
>>> Decompression of fz files complete.

2023-02-26 09:36:45 - Decompressing and splitting data files...
>>> Starting decompression of bz2 files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230226_geminiS/rawfits_r/...
>>> Decompression of bz2 files complete.
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230226_geminiS/rawfits_r/...
>>> Decompression of fz f

>>> Overscan correction and trimming complete.
Starting bias subtraction...
>>> Bias subtraction complete.
>>> Starting adjacent amp area concatenation...
>>> Adjacent amp area concatenation complete.
Starting flatfield correction...
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead

In [7]:
#trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-Q-111/ut20221023_geminiS/flatfits_pyt/c01_432p/')
#trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-Q-111/ut20221023_geminiS/flatfits_pyt/c02_p2016j1b/')
#trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-Q-111/ut20221027_geminiS/flatfits_pyt/c01_324p/')
#trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-Q-111/ut20221118_geminiS/flatfits_pyt/c01_238p/')
#trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2022B-Q-111/ut20230122_geminiS/flatfits_pyt/')

trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230213_geminiS/flatfits_pyt/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230215_geminiS/flatfits_pyt/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230216_geminiS/flatfits_pyt/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230217_geminiS/flatfits_pyt/c01_133p/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230217_geminiS/flatfits_pyt/c02_439p/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230217_geminiS/flatfits_pyt/c03_p2021a5/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230219_geminiS/flatfits_pyt/c01_348p/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230219_geminiS/flatfits_pyt/c02_435p/')
trim_gmoss_data_with_bad_amp('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2023A-LP-104/ut20230226_geminiS/flatfits_pyt/')


Trimming GMOS-S data with bad amp #5...
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the 